In [1]:
import pandas as pd
import numpy as np
import random

In [1]:
import random
import pandas as pd

subjects = ["Math", "Science", "English", "History", "Programming"]
difficulty_levels = {"Math": 4, "Science": 3, "English": 2, "History": 2, "Programming": 5}

data = []

for subject in subjects:
    for _ in range(50):
        performance = random.randint(40, 95)
        days_left = random.randint(5, 30)
        difficulty = difficulty_levels[subject]

        if performance < 60:
            base_hours = random.randint(10, 20)
        elif performance < 80:
            base_hours = random.randint(5, 10)
        else:
            base_hours = random.randint(2, 5)

        urgency_factor = max(1, (30 - days_left) / 10)
        recommended_hours = round(base_hours * difficulty * urgency_factor / 5, 1)

        if recommended_hours > 10:
            priority = "High"
        elif recommended_hours > 5:
            priority = "Medium"
        else:
            priority = "Low"

        data.append({
            "Subject": subject,
            "Difficulty": difficulty,
            "Performance_Score": performance,
            "Days_Left": days_left,
            "Recommended_Hours": recommended_hours,
            "Priority_Level": priority
        })

df = pd.DataFrame(data)

print(df.head(10))

df.to_csv("clean_study_planner_dataset.csv", index=False)


  Subject  Difficulty  Performance_Score  Days_Left  Recommended_Hours  \
0    Math           4                 51         23               16.0   
1    Math           4                 44         10               16.0   
2    Math           4                 71         23                6.4   
3    Math           4                 70         13               13.6   
4    Math           4                 85         29                1.6   
5    Math           4                 57         24               16.0   
6    Math           4                 43         29               14.4   
7    Math           4                 52         29               10.4   
8    Math           4                 42          6               19.2   
9    Math           4                 55          8               21.1   

  Priority_Level  
0           High  
1           High  
2         Medium  
3           High  
4            Low  
5           High  
6           High  
7           High  
8           Hi

In [24]:
X = df.drop('Recommended_Hours', axis = 1)
Y = df['Recommended_Hours']

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [26]:
df.head()

,Subject,Difficulty,Performance_Score,Days_Left,Recommended_Hours,Priority_Level
0,Math,4,51,23,16.0,High
1,Math,4,44,10,16.0,High
2,Math,4,71,23,6.4,Medium
3,Math,4,70,13,13.6,High
4,Math,4,85,29,1.6,Low


In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline

In [28]:
ohe = OneHotEncoder()
ohe.fit(X[['Subject']])

,categories,'auto'
,drop,None
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [29]:
df['Priority_Level'].unique()

array(['High', 'Medium', 'Low'], dtype=object)

In [30]:
trf = ColumnTransformer(transformers = [
    ('ohe', OneHotEncoder(categories = ohe.categories_), [0]),
    ('oe', OrdinalEncoder(categories = [['Low', 'Medium', 'High']]), [4])
], remainder = 'passthrough')

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

In [32]:
lr = LinearRegression()
dt = DecisionTreeRegressor()
bg = BaggingRegressor()
rf = RandomForestRegressor()
gbr = GradientBoostingRegressor()

In [38]:
pipe = make_pipeline(trf, gbr)
pipe.fit(X_train, Y_train)
r2_score(Y_test, pipe.predict(X_test))

0.8683205825307323

In [40]:
import pickle
with open('study_planner_model.pkl', 'wb') as f:
    pickle.dump(pipe, f)

In [42]:
df['Subject'].unique()

array(['Math', 'Science', 'English', 'History', 'Programming'],
      dtype=object)